[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/keras-team/autokeras/blob/master/docs/templates/tutorial/image_classification.ipynb)

In [1]:
import tensorflow as tf
import autokeras as ak
tf.__version__

'2.1.0'

## A Simple Example

### Load MNIST dataset

In [2]:
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print('Training image shape:', x_train.shape) # (60000, 28, 28)
print('Training label shape:', y_train.shape) # (60000,)
print('First five training labels:', y_train[:5]) # array([5 0 4 1 9], dtype=uint8)

Training image shape: (60000, 28, 28)
Training label shape: (60000,)
First five training labels: [5 0 4 1 9]


### Run the ImageClassifier

In [3]:
# Initialize the image classifier.
clf = ak.ImageClassifier(max_trials=2) # It tries two different models.

# Feed the image classifier with training data 
# 20% of the data is used as validation data by default for tuning
# the process may run for a bit long time, please try to use GPU
clf.fit(x_train, y_train, epochs=3, verbose=2, ) # each model is trained for three epochs

Train for 1500 steps, validate for 375 steps
Epoch 1/3
1500/1500 - 20s - loss: 0.1804 - accuracy: 0.9447 - val_loss: 0.0720 - val_accuracy: 0.9783
Epoch 2/3
1500/1500 - 15s - loss: 0.0798 - accuracy: 0.9756 - val_loss: 0.0533 - val_accuracy: 0.9851
Epoch 3/3
1500/1500 - 14s - loss: 0.0634 - accuracy: 0.9809 - val_loss: 0.0484 - val_accuracy: 0.9856


Train for 1500 steps, validate for 375 steps
Epoch 1/3
1500/1500 - 90s - loss: 0.2320 - accuracy: 0.9350 - val_loss: 0.1785 - val_accuracy: 0.9456
Epoch 2/3
1500/1500 - 74s - loss: 0.1139 - accuracy: 0.9707 - val_loss: 0.7658 - val_accuracy: 0.7822
Epoch 3/3
1500/1500 - 77s - loss: 0.0883 - accuracy: 0.9761 - val_loss: 0.0832 - val_accuracy: 0.9757


INFO:tensorflow:Oracle triggered exit
Train for 1875 steps, validate for 375 steps
Epoch 1/3
1875/1875 - 17s - loss: 0.1557 - accuracy: 0.9521 - val_loss: 0.0420 - val_accuracy: 0.9869
Epoch 2/3
1875/1875 - 17s - loss: 0.0742 - accuracy: 0.9774 - val_loss: 0.0242 - val_accuracy: 0.9923
Epoch 3/3
1875/1875 - 17s - loss: 0.0559 - accuracy: 0.9829 - val_loss: 0.0175 - val_accuracy: 0.9949


### Get the summarized results during the tuning process (return the best 10 models if existed)

In [7]:
clf.tuner.results_summary()

### Retrieve best model

In [21]:
best_model = clf.export_model()
best_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
normalization (Normalization (None, 28, 28, 1)         3         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0     


### Predict with the best model.


In [22]:
predicted_y = clf.predict(x_test)
print(predicted_y)

[[7]
 [2]
 [1]
 ...
 [4]
 [5]
 [6]]


### Evaluate the best model on the test data.

In [15]:

test_loss, test_acc = clf.evaluate(x_test, y_test, verbose=0)
print('Test accuracy: ', test_acc)


Test accuracy:  0.9878


## Validation Data

By default, AutoKeras use the last 20% of training data as validation data. As shown in the example below, you can use validation_split to specify the percentage.

In [ ]:
clf.fit(x_train,
        y_train,
        # Split the training data and use the last 15% as validation data.
        validation_split=0.15,epochs=3)

You can also use your own validation set instead of splitting it from the training data with validation_data.

In [ ]:
split = 50000
x_val = x_train[split:]
y_val = y_train[split:]
x_train = x_train[:split]
y_train = y_train[:split]
clf.fit(x_train,
        y_train,
        # Use your own validation set.
        validation_data=(x_val, y_val),epochs=3)

## Data Format

The AutoKeras ImageClassifier is quite flexible for the data format.

For the image, it accepts data formats both with and without the channel dimension. The images in the MNIST dataset do not have the channel dimension. Each image is a matrix with shape (28, 28). AutoKeras also accepts images of three dimensions with the channel dimension at last, e.g., (32, 32, 3), (28, 28, 1).

For the classification labels, AutoKeras accepts both plain labels, i.e. strings or integers, and one-hot encoded encoded labels, i.e. vectors of 0s and 1s.

So if you prepare your data in the following way, the ImageClassifier should still work.

In [5]:
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Reshape the images to have the channel dimension.
# x_train = x_train.reshape(x_train.shape + (1,))
# x_test = x_test.reshape(x_test.shape + (1,))

x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]


# One-hot encode the labels.
import numpy as np
eye = np.eye(10)
y_train = eye[y_train]
y_test = eye[y_test]

print(x_train.shape) # (60000, 28, 28, 1)
print(y_train.shape) # (60000, 10)
print(y_train[:3])
# array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
#        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
#        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]])

(60000, 28, 28, 1)
(60000, 10)
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]


We also support using tf.data.Dataset format for the training data. In this case, the images would have to be 3-dimentional. The labels have to be one-hot encoded for multi-class classification to be wrapped into tensorflow Dataset.

In [7]:
import tensorflow as tf
train_set = tf.data.Dataset.from_tensor_slices(((x_train, ), (y_train, )))
test_set = tf.data.Dataset.from_tensor_slices(((x_test, ), (y_test, )))

clf = ak.ImageClassifier(max_trials=10)
# Feed the tensorflow Dataset to the classifier.
clf.fit(train_set)
# Predict with the best model.
predicted_y = clf.predict(test_set)
# Evaluate the best model with testing data.
print(clf.evaluate(test_set))

TypeError: '>' not supported between instances of 'TensorShape' and 'int'

### Configurate search process

In [5]:
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

clf = ak.ImageClassifier(max_trials=2, 
                         loss='categorical_crossentropy', 
                         metrics=['accuracy'],
                         objective='val_accuracy',
                        )

clf.fit(x_train, y_train, 
        validation_split=0.15,
        epochs=3, verbose=2, )

Train for 1500 steps, validate for 375 steps
Epoch 1/3
1500/1500 - 24s - loss: 0.1769 - accuracy: 0.9466 - val_loss: 0.0683 - val_accuracy: 0.9804
Epoch 2/3
1500/1500 - 18s - loss: 0.0795 - accuracy: 0.9753 - val_loss: 0.0546 - val_accuracy: 0.9849
Epoch 3/3
1500/1500 - 19s - loss: 0.0637 - accuracy: 0.9800 - val_loss: 0.0499 - val_accuracy: 0.9855


Train for 1500 steps, validate for 375 steps
Epoch 1/3
1500/1500 - 120s - loss: 0.2389 - accuracy: 0.9338 - val_loss: 1.1822 - val_accuracy: 0.8832
Epoch 2/3
1500/1500 - 99s - loss: 0.1327 - accuracy: 0.9673 - val_loss: 0.0650 - val_accuracy: 0.9836
Epoch 3/3
1500/1500 - 99s - loss: 0.0719 - accuracy: 0.9802 - val_loss: 0.3224 - val_accuracy: 0.9469


INFO:tensorflow:Oracle triggered exit
Train for 1875 steps, validate for 375 steps
Epoch 1/3
1875/1875 - 24s - loss: 0.1593 - accuracy: 0.9518 - val_loss: 0.0523 - val_accuracy: 0.9833
Epoch 2/3
1875/1875 - 22s - loss: 0.0708 - accuracy: 0.9779 - val_loss: 0.0350 - val_accuracy: 0.9885
Epoch 3/3
1875/1875 - 22s - loss: 0.0605 - accuracy: 0.9814 - val_loss: 0.0218 - val_accuracy: 0.9937


## Reference
[ImageClassifier](/image_classifier),
[AutoModel](/auto_model/#automodel-class),
[ImageBlock](/block/#imageblock-class),
[Normalization](/preprocessor/#normalization-class),
[ImageAugmentation](/preprocessor/#image-augmentation-class),
[ResNetBlock](/block/#resnetblock-class),
[ImageInput](/node/#imageinput-class),
[ClassificationHead](/head/#classificationhead-class).